In [ ]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

In [ ]:
import pm4py
from pm4py.objects.dcr import semantics_obj as dcr_sema
from pm4py.algo.discovery.dcr_discover import algorithm as disc_alg
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter

# File names

In [ ]:
event_log_file = '/home/vco/Datasets/Sepsis Cases - Event Log.xes'
result_file_prefix = 'sepsis_dcr'
dcrxml_file_export = 'models/sepsis_dcr.xml'
dcrxml_file_import = 'models/sepsis_dcr_from_portal.xml'
dcr_title = 'Sepsis Cases'

# Mine DCR graph using the DisCoveR miner

In [ ]:
log = pm4py.read_xes(event_log_file)
dcr, _ = disc_alg.apply(log)
dcr_exporter.apply(dcr_graph=dcr,path=dcrxml_file_export,variant=dcr_exporter.DCR_XML_SIMPLE)
# Import this to the portal with the appstore installed "Import DCR Process Model" button

# Play with the execution semantics
(note that no-responses are not supported yet)

In [ ]:
# Export the file from the portal and save it in the models folder as sepsis_dcr_from_portal.xml
# Now we import it in pm4py
dcr = dcr_importer.apply(path=dcrxml_file_import,variant=dcr_importer.DEFAULT_VARIANT)

In [ ]:
dcr_semantics = dcr_sema.DcrSemantics(dcr)

In [ ]:
print(dcr_semantics.enabled()) # check enabled events

In [ ]:
event_to_execute = 'CRP' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute) # here you execute the event
print(dcr_semantics.enabled()) # now these are the newly enabled events

In [ ]:
dcr_semantics.is_accepting()

In [ ]:
event_to_execute = 'ER Sepsis Triage' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute) # here you execute the event
print(dcr_semantics.enabled()) # now these are the newly enabled events

In [ ]:
dcr_semantics.is_accepting() # this checks the acceptance criteria for the dcr graph